# 1. Importar bibliotecas


In [1]:

import os
import re
import pandas as pd
import pdfplumber
from datetime import datetime
import logging
import shutil
import time


Suprime o warning "CropBox missing from /Page"

In [19]:
logging.getLogger('pdfminer').setLevel(logging.ERROR)

# 2. Definir o diretório de trabalho e nome do arquivo de saída

Define o diretório atual onde estão os arquivos PDF

In [20]:
pasta_pdf = os.getcwd()

Define o nome do arquivo de saída Excel

In [21]:
arquivo_saida = os.path.join(pasta_pdf, 'extrato_nu_cartao.xlsx')

# 3. Criar um backup do arquivo existente (se houver)


In [25]:
if os.path.exists(arquivo_saida):
    backup_path = arquivo_saida.replace('.xlsx', f'_backup {datetime.now().strftime("%Y.%m.%d - %H.%M.%S")}.xlsx')
    shutil.copy(arquivo_saida, backup_path)
    print(f'Backup criado: {backup_path}')


Backup criado: c:\Users\rodri\OneDrive\Documentos\GitHub\Minhas-Finanças\dados\nu\cartao\extrato_nu_cartao_backup 2025.07.12 - 20.26.12.xlsx



# 4. Carregar dados existentes e identificar competências já importadas


In [26]:
if os.path.exists(arquivo_saida):
    df_existente = pd.read_excel(arquivo_saida)
    competencias_existentes = set(df_existente['Competência'].unique())
else:
    df_existente = pd.DataFrame()
    competencias_existentes = set()


Exibir arquivo existente

In [27]:
if not df_existente.empty:
    print(df_existente.head())

        Data Competência              Descrição Receita/Despesa  Valor
0 2022-01-07     2022-02    Pagamento em 07 JAN         Despesa  20.04
1 2022-01-08     2022-02       Htm*Mepoupe1real         Despesa   1.00
2 2022-01-08     2022-02           Pag*Acailand         Despesa  19.04
3 2022-01-14     2022-02  Farmacias Pague Menos         Despesa  21.70
4 2022-01-18     2022-02           Uhuu *785483         Despesa  80.50



# 5. Inicializar a extração de dados


Dataframe para gravar dados

In [28]:
todos_dados = []

Mapeamento de meses abreviados para números

In [29]:
meses = {
    'JAN': '01', 'FEV': '02', 'MAR': '03', 'ABR': '04',
    'MAI': '05', 'JUN': '06', 'JUL': '07', 'AGO': '08',
    'SET': '09', 'OUT': '10', 'NOV': '11', 'DEZ': '12'
}

Expressão regular para identificar linhas de transações

In [30]:
padrao_linha = re.compile(r'^(\d{2})\s+([A-Z]{3})\s+(.+?)\s+([\d.,\s]+)$')

# 6. Iterar sobre os arquivos PDF e extrair dados

In [31]:
for arquivo in os.listdir(pasta_pdf):

    if arquivo.endswith('.pdf') and arquivo.startswith('Nubank_'):
        match = re.search(r'Nubank_(\d{4})[_-](\d{2})', arquivo)
        if not match:
            print(f'Nome de arquivo inesperado: {arquivo}')
            continue

        ano, mes = match.groups()
        competencia = f'{ano}-{mes}'

        if competencia in competencias_existentes:
            print(f'Ignorado (já importado): {arquivo}')
            continue

        caminho = os.path.join(pasta_pdf, arquivo)

        with pdfplumber.open(caminho) as pdf:
            comecar = False
            for pagina in pdf.pages:
                try:
                    texto = pagina.extract_text()
                except Exception:
                    continue

                if not texto:
                    continue

                linhas = texto.split('\n')
                for linha in linhas:
                    if not comecar:
                        if linha.startswith('TRANSAÇÕES DE '):
                            comecar = True
                        continue

                    match = padrao_linha.match(linha.strip())
                    if match:
                        dia, mes_str, descricao, valor = match.groups()
                        mes_num = meses.get(mes_str.upper())
                        if not mes_num:
                            continue

                        data_str = f'{ano}-{mes_num}-{dia}'
                        data_formatada = datetime.strptime(data_str, '%Y-%m-%d').date()
                        valor_float = float(valor.replace('.', '').replace(',', '.').strip())
                        descricao = descricao.strip()

                        receita_keywords = ['Estorno de', 'Crédito de Confiança de', 'Pagamento recebido', 'Transferência recebida']
                        tipo = 'Receita' if any(descricao.startswith(k) for k in receita_keywords) else 'Despesa'

                        todos_dados.append({
                            'Data': data_formatada,
                            'Competência': competencia,
                            'Descrição': descricao,
                            'Receita/Despesa': tipo,
                            'Valor': valor_float
                        })
        print(f'Processado: {arquivo}')


Ignorado (já importado): Nubank_2022-02-09.pdf
Ignorado (já importado): Nubank_2022-03-09.pdf
Ignorado (já importado): Nubank_2022-04-09.pdf
Ignorado (já importado): Nubank_2022-05-09.pdf
Ignorado (já importado): Nubank_2022-06-09.pdf
Ignorado (já importado): Nubank_2022-07-09.pdf
Ignorado (já importado): Nubank_2022-08-09.pdf
Ignorado (já importado): Nubank_2022-09-09.pdf
Ignorado (já importado): Nubank_2022-10-09.pdf
Ignorado (já importado): Nubank_2022-11-09.pdf
Ignorado (já importado): Nubank_2022-12-09.pdf
Ignorado (já importado): Nubank_2023-01-09.pdf
Ignorado (já importado): Nubank_2023-02-09.pdf
Ignorado (já importado): Nubank_2023-03-09.pdf
Ignorado (já importado): Nubank_2023-04-09.pdf
Ignorado (já importado): Nubank_2023-06-09.pdf
Ignorado (já importado): Nubank_2023-07-09.pdf
Ignorado (já importado): Nubank_2023-08-09.pdf
Ignorado (já importado): Nubank_2023-09-09.pdf
Ignorado (já importado): Nubank_2023-10-09.pdf
Ignorado (já importado): Nubank_2023-11-09.pdf
Ignorado (já 

# 7. Consolidar dados

In [32]:
if not todos_dados:
    print('Nenhum novo dado foi importado.')
else:
    df_novos = pd.DataFrame(todos_dados)
    df_novos = df_novos.sort_values(by='Data')

    if not df_existente.empty:
        df_final = pd.concat([df_existente, df_novos], ignore_index=True)
    else:
        df_final = df_novos

    # Valores absolutos (positivos)
    df_final['Valor'] = df_final['Valor'].abs()

Nenhum novo dado foi importado.


Exibir dados processados

In [33]:
if not df_final.empty:
    print(df_final.head())

         Data Competência              Descrição Receita/Despesa  Valor
0  2022-01-07     2022-02    Pagamento em 07 JAN         Despesa  20.04
1  2022-01-08     2022-02       Htm*Mepoupe1real         Despesa   1.00
2  2022-01-08     2022-02           Pag*Acailand         Despesa  19.04
3  2022-01-14     2022-02  Farmacias Pague Menos         Despesa  21.70
4  2022-01-18     2022-02           Uhuu *785483         Despesa  80.50


# 8. Ordenar e Salvar Excel

In [34]:
df_final = df_final.sort_values(by='Data')
df_final.to_excel(arquivo_saida, index=False, sheet_name='Extrato')
print(f'Dados salvos em: {arquivo_saida}')
print(f'Total de registros no arquivo: {len(df_final)}')
print(f'Novos registros adicionados: {len(df_novos)}')


Dados salvos em: c:\Users\rodri\OneDrive\Documentos\GitHub\Minhas-Finanças\dados\nu\cartao\extrato_nu_cartao.xlsx
Total de registros no arquivo: 1412
Novos registros adicionados: 1412
